## Local Functions

In [1]:
def sort_by_last_letter(strings):
    
    def last_letter(string):
        return string[-1]
    
    return sorted(strings, key = last_letter)


In [6]:
words = ["deepak", "gaurav", "jai"]
sort_by_last_letter(words)

['jai', 'deepak', 'gaurav']

In [8]:
sort_by_last_letter.last_letter
# the new function(last_letter()) is created after each time def is executed

AttributeError: 'function' object has no attribute 'last_letter'

In [10]:
g = "global"
def outer(p = "params"):
    l = "local"
    def local():
        print(l, g, p)
    
    local()
    
outer()


local global params


In [14]:
x = "global"
print(x)
def outer(x = "params"):
    print(x)
    x = "local"
    def local():
        print(x)
    
    local()
    
outer()

global
params
local


### Returning local functions

In [15]:
def outer():
    def inner():
        print("inner")
    return inner

inner_func = outer()
inner_func()

# How?
# Because of CLOSURES property of python. which maintains reference to objects from local scopes

inner


## Closures

In [23]:
def enclosing():
    x = "closed over"
    b = True
    d = {"name": "jai", "age": 21}
    y = 2 # not in closurebecause not used in inner function
    
    def inner_function():
        print(x, b, d)
    
    return inner_function

lf = enclosing()
lf()
lf.__closure__

closed over True {'age': 21, 'name': 'jai'}


(<cell at 0x7f716cd12438: bool object at 0x7f717a659540>,
 <cell at 0x7f716cd120a8: dict object at 0x7f716cca2cc8>,
 <cell at 0x7f716cd12618: str object at 0x7f716cd11370>)

In [27]:
def raise_to(exp):
    def raise_to_exp(x):
        return exp**x
    return raise_to_exp

x = raise_to(6)
print(x(2))
print(x(0))
print(x(6))
print(x(1))
x.__closure__

36
1
46656
6


(<cell at 0x7f716cd12888: int object at 0x7f717a6bbac0>,)

### nonlocal vs global

In [34]:
message = "global"

def enclosing():
    message = "enclosing"
    
    def local():
        message = "local"
    
    print("enclosing function: ", message)
    local()
    print("enclosing function: ", message)

print("global message: ", message)
enclosing()
print("global message: ", message)

global message:  global
enclosing function:  enclosing
enclosing function:  enclosing
global message:  global


In [35]:
message = "global"

def enclosing():
    message = "enclosing"
    
    def local():
        global message
        message = "local"

    print("enclosing function: ", message)
    local()
    print("enclosing function: ", message)

print("global message: ", message)
enclosing()
print("global message: ", message)

global message:  global
enclosing function:  enclosing
enclosing function:  enclosing
global message:  local


In [36]:
message = "global"

def enclosing():
    message = "enclosing"
    
    def local():
        nonlocal message
        message = "local"

    print("enclosing function: ", message)
    local()
    print("enclosing function: ", message)

print("global message: ", message)
enclosing()
print("global message: ", message)

global message:  global
enclosing function:  enclosing
enclosing function:  local
global message:  global


# Decorators

modify or enhance the functions withour changing their deinations

##### implemeted as callables that take and return other callable

- Replace, enhance,  or modify existing function
- Does not change the orignal function defination
- Calling code does need not to change
- Decorator mechanism uses the modified function's orignal name

It first compliles the base function(creates a Function object) and then passes this fncyion object to the decorator.

Decorator by defination takes arguement as function object as only arguement, and it returns a new callable object which is nothing but a new local function defined inside the decorator

And then function binds with the orignal function name.

### Function as Callables

In [10]:
def escape_unicode(f):
    print("2")
    def wrap(*args, **kwargs):
        print("3")
        x = f(*args, **kwargs)
        print("4")
        return ascii(x)
    print("5")
    return wrap

def northen_city():
    return "aα"

@escape_unicode
def northen_city1():
    print("1")
    return "aα"

2
5


In [6]:
northen_city()

'aα'

In [11]:
northen_city1()

3
1
4


"'a\\u03b1'"

### Classes as Callable

Need to have \__call__ method

In [26]:
# Call count, Class as Decorator(only if it as __Call__ method)

class CallCount:
    def __init__(self, f):
        self.count = 0
        self.f = f
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        self.f(*args, **kwargs)
        return self.f(*args, **kwargs)

@CallCount
def hello(name):
    return "Hello, {}".format(name)


In [27]:
hello("Jai")

'Hello, Jai'

In [28]:
hello("Deepak")

'Hello, Deepak'

In [29]:
hello.count

2

### Instance as Decorator



In [59]:
class Trace:
    def __init__(self):
        self.enabled = True
    
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self.enabled:
                print("Calling {}".format(f))
            return f(*args, **kwargs)
        return wrap

tracer = Trace()

In [47]:
@tracer
def rotate_list(l):
    return l[1:] + [l[0]]

In [52]:
l = [1, 2, 3]
m = rotate_list(l)

Calling <function rotate_list at 0x7f22803f46a8>


In [56]:
m = rotate_list(l)

Calling <function rotate_list at 0x7f22803f46a8>


In [57]:
m = rotate_list(l)

Calling <function rotate_list at 0x7f22803f46a8>


In [60]:
tracer.enabled = False
m = rotate_list(l)

## Multiple Decorator

It first applies the decorator3 and new returned function is then passed to decorator2 and then to 1.

In [64]:
tracer.enabled = True


@tracer
@escape_unicode
def nor_weign_island_maker(name):
    return name + "deltaΔ"

2
5


In [65]:
nor_weign_island_maker("Jai")

Calling <function escape_unicode.<locals>.wrap at 0x7f22803f9488>
3
4


"'Jaidelta\\u0394'"

In [66]:
tracer.enabled = False
nor_weign_island_maker("Jai")

3
4


"'Jaidelta\\u0394'"

### Functools.wrap()
During applying decorators, we lose the meta data of the function which we have applied on the decorator.
Or it overrided by the decorator meta data

In [67]:
# FOr EXAMPLE

def noop(f):
    def wrap(*args, **kwargs):
        return f(*args, **kwargs)
    return wrap

@noop
def hello():
    """Prints the hello world message"""
    print("Hello world!!")

In [68]:
hello.__name__

'wrap'

In [70]:
hello.__doc__

In [72]:
help(hello)

Help on function wrap in module __main__:

wrap(*args, **kwargs)



##### To avoid this losing of Meta Data, we use functools.wrap()

In [78]:
from functools import wraps

def noop(f):
    
    @wraps(f)
    def wrap(*args, **kwargs):
        return f(*args, **kwargs)
    
    return wrap

@noop
def hello():
    """Prints the hello world message"""
    print("Hello world!!")

In [79]:
hello.__name__

'hello'

In [81]:
hello.__doc__

'Prints the hello world message'

In [82]:
help(hello)

Help on function hello in module __main__:

hello()
    Prints the hello world message

